<a href="https://colab.research.google.com/github/bhargav23/EVA4/blob/master/S4/EVA4_Session_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1) #input 28x28x1, output 28x28x16, RF 3
        self.bn1 = nn.BatchNorm2d(16)
        self.do1 = nn.Dropout(0.15)

        self.conv2 = nn.Conv2d(16, 32, 3, padding=1) #input 28x28x16, output 28x28x32, RF 5
        self.bn2 = nn.BatchNorm2d(32)
        self.do2 = nn.Dropout(0.15)

        self.pool1 = nn.MaxPool2d(2, 2) # input 28x28*32 output 14*14*32 RF 6

        self.conv3 = nn.Conv2d(32, 16, 3, padding=1) #input 14x14x32, output 14x14x16, RF 10
        self.bn3 = nn.BatchNorm2d(16)
        self.do3 = nn.Dropout(0.15)

        self.conv4 = nn.Conv2d(16, 16, 3, padding=1) #input 14*14*16 output 14*14*16 RF 14
        self.bn4 = nn.BatchNorm2d(16)
        self.do4 = nn.Dropout(0.1)

        self.pool2 = nn.MaxPool2d(2, 2) # input 14*14*16 op 7*7*16 RF 16

        self.conv5 = nn.Conv2d(16, 16, 3) # input 7*7*16 op 7*7*16 RF 24
        self.bn5 = nn.BatchNorm2d(16)
        self.do5 = nn.Dropout(0.1)

        self.conv6 = nn.Conv2d(16, 16, 3) # input 5*5*16 op 5*5*16 RF 32
        self.bn6 = nn.BatchNorm2d(16)
        self.do6 = nn.Dropout(0.05)

        self.conv7 = nn.Conv2d(16, 10, 3) # input 3*3*16 op 1*1*10 RF 40

    def forward(self, x):


        x = self.pool1(F.relu(self.bn2(self.do2(self.conv2(F.relu(self.bn1(self.do1(self.conv1(x)))))))))
        x = self.pool2(F.relu(self.bn4(self.do4(self.conv4(F.relu(self.bn3(self.do3(self.conv3(x)))))))))
        x = F.relu(self.bn6(self.do6(self.conv6(self.bn5(self.do5(self.conv5(x)))))))
        x = self.conv7(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [10]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             160
           Dropout-2           [-1, 16, 28, 28]               0
       BatchNorm2d-3           [-1, 16, 28, 28]              32
            Conv2d-4           [-1, 32, 28, 28]           4,640
           Dropout-5           [-1, 32, 28, 28]               0
       BatchNorm2d-6           [-1, 32, 28, 28]              64
         MaxPool2d-7           [-1, 32, 14, 14]               0
            Conv2d-8           [-1, 16, 14, 14]           4,624
           Dropout-9           [-1, 16, 14, 14]               0
      BatchNorm2d-10           [-1, 16, 14, 14]              32
           Conv2d-11           [-1, 16, 14, 14]           2,320
          Dropout-12           [-1, 16, 14, 14]               0
      BatchNorm2d-13           [-1, 16, 14, 14]              32
        MaxPool2d-14             [-1, 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:42: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [13]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 21):
    print(epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]

1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:42: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0882, Accuracy: 9769/10000 (97.69%)

2


  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0202, Accuracy: 9941/10000 (99.41%)

19


  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0180, Accuracy: 9944/10000 (99.44%)

20


loss=0.002100229263305664 batch_id=468: 100%|██████████| 469/469 [00:11<00:00, 44.49it/s]



Test set: Average loss: 0.0187, Accuracy: 9930/10000 (99.30%)

